#### calc RRCS from two tensor ([14,3] [14,3])

In [ ]:
    
from typing import Optional
import torch
from Bio.PDB.PDBParser import PDBParser 
# from utils.residue_constants import *
from multi_state.utils.calc_LDDT import parse_pdb
res1 = [[1, 2, 3],[2, 2, 3],[3, 23, 3],[0,0,0]]
res2 = [[1, 3, 3],[2, 3, 3],[3, 32, 3],[0,0,0]]
res1 = torch.tensor(res1, dtype=torch.long)
res2 = torch.tensor(res2, dtype=torch.long)
def calc_2res_RRCS(res1: torch.tensor, res2: torch.tensor) -> float:
    """calculate RRCS within two residues

    Args:
        res1 (torch.tensor): shape(14, 3)
        res2 (torch.tensor): shape(14, 3)

    Returns:
        score: tensor
    """
    res1_del = res1[torch.max(res1[:]>0,1)[0]] # del row=0
    res2_del = res2[torch.max(res2[:]>0,1)[0]] # del row=0
    res1_2 = (res1_del ** 2).sum(dim=1).reshape((-1, 1)) #calc res1**2
    res2_2 = (res2_del ** 2).sum(dim=1)
    D = torch.clamp((res1_2 + res2_2 - 2 * res1_del.mm(res2_del.t()))**0.5,min=3.23,max=4.63) #calc dist
    score = 1. - (D - 3.23) / 1.40
    return float(score.mean())
p_f = "/home/yangsy/projects/multi_state/raw_leftpdbs/2Y04.pdb"
pos = torch.tensor(parse_pdb(p_f,"B"))
# TODO: generate residues index from GN
def calc_pdb_RRCS(pos: torch.tensor) -> torch.tensor:
    """calculate RRCS between all residues in a structure

    Args:
        pos (torch.tensor): [N, 14, 3]

    Returns:
        score (torch.tensor): [N, N] eg. score[1,298] means the score between res No.2 & No.299, so ==score[298,1]
    """
    # norm means L2; add none dim can calc all to all RRCS; clamp means turn number into [3.23,4.63]
    all2all_dist = torch.clamp((pos[..., :, None, :, None, :] - pos[..., None, :, None, :, :]).norm(dim=-1),
                    min=3.23, max=4.63)
    # get residue atom mask, avoid adding score between none-zero 3D and zero 3D(no atoms in fact) 
    nonzeros = torch.where(pos != 0, torch.ones_like(pos), pos) 
    mask = (nonzeros[..., :, None, :, None, :] * nonzeros[..., None, :, None, :, :]).mean(-1)
    score = (1. - (all2all_dist - 3.23) / 1.40) * mask
    score = score.sum((-1,-2)) # sum atom-atom score to a residue. [N,N,14,14] -> [N,N]
    return score 

score = calc_pdb_RRCS(pos)
print(score)



#### 字符串匹配

In [7]:
import difflib
from typing import Tuple, List
s1 = "----------THRMRTETKAAKTLCIIMGCFCL-CWAPFFVTNIVDPFI----"
s2 = "KVVLLTFLSTVILMAILGNLLVMVAVCWDRQLRKIKTNYFIVSLAFADLLVSVLVMPFGAIELVQDIWIYGEVFCLVRTSLDVLLTTASIFHLCCISLDRYYAICCQPLVYRNKMTPLRIALMLGGCWVIPTFISFLPIMQGWNNIGIIDLIEKRKFNQNSNSTYCVFMVNKPYAITCSVVAFYIPFLLMVLAYYRIYVTAKEHAHQIQMLQRAGASSETKAAKTLCIIMGCFCLCWAPFFVTNIVDPFIDYTVPGQVWTAFLWLGYINSGLNPFLYAFLNKSFRRAFLIILCC"
ls = 0
for s in s1:
    if s != "-":
        break
    else:
        ls += 1
rs = 0
for s in s1[::-1]:
    if s != "-":
        break
    else:
        rs += 1
s1_new = s1.strip("-")
ls1 = len(s1_new)
s2_new = "#" * ls1 + s2 + "#" * ls1
ls2 = len(s2_new)
n_b = s1_new.count("-")
ratios = [[] for _ in range(n_b + 1)]
print(s2_new[lb:lb + ls1 - n_bi])
for n_bi in range(n_b + 1):
    for lb in range(ls2 - ls1 + 1):
        ratios[n_bi].append(difflib.SequenceMatcher(None, s1_new, s2_new[lb:lb + ls1 - n_bi]).quick_ratio())
        print(ratios[n_bi])
        print(s2_new[lb:lb + ls1 - n_bi])
        # print(difflib.SequenceMatcher(None, s1_new, s2_new[lb:lb + ls1 - n_bi]).quick_ratio())
m_ij = [max(i) for i in ratios]
m_i = max(m_ij)
id_i = m_ij.index(m_i)
id_j = ratios[id_i].index(m_i)
select_s2_p = s2_new[id_j:id_j + ls1 - id_i]

s1_ids = list(range(ls1))
s2_ids = [-1] * (ls1 - id_i)
# TODO: Because the transform only delete "-" and insert "-", so cannot use get_opcodes() methods.
matchs = difflib.SequenceMatcher(None, s1_new, select_s2_p).get_matching_blocks()
for m in matchs:
    if m.size > 2:
        s2_ids[m.b:m.b + m.size] = s1_ids[m.a:m.a + m.size]
# TODO: set initial
for id, i in enumerate(s2_ids):
    if i == -1:
        s2_ids[id] = id
    else:
        break
# shift
for id, i in enumerate(s2_ids):
    s2_ids[id] = i + ls
s2_ids = ls * [-1] + s2_ids + rs * [-1]
start = id_j - ls1 - ls
match_ids = s2_ids
match = (s1, s2[start:start + len(s2_ids)])
match_core = (s1_new, select_s2_p)
ratio = m_i

######################################
[0.0]
#######################################
[0.0, 0.02564102564102564]
######################################K
[0.0, 0.02564102564102564, 0.05128205128205128]
#####################################KV
[0.0, 0.02564102564102564, 0.05128205128205128, 0.07692307692307693]
####################################KVV
[0.0, 0.02564102564102564, 0.05128205128205128, 0.07692307692307693, 0.10256410256410256]
###################################KVVL
[0.0, 0.02564102564102564, 0.05128205128205128, 0.07692307692307693, 0.10256410256410256, 0.1282051282051282]
##################################KVVLL
[0.0, 0.02564102564102564, 0.05128205128205128, 0.07692307692307693, 0.10256410256410256, 0.1282051282051282, 0.15384615384615385]
#################################KVVLLT
[0.0, 0.02564102564102564, 0.05128205128205128, 0.07692307692307693, 0.10256410256410256, 0.1282051282051282, 0.15384615384615385, 0.1794871794871795]
################################KVVLLTF
[0.0, 0.0

In [ ]:
import json
def getmask(PDBid):
    TM3 = ['3x43', '3x46', '3x50']
    TM6 = ['6x40', '6x41', '6x37']
    TM7 = ['7x49', '7x53']
    with open("all_res_pos.json","r") as f:
        GN_dict = json.load(f)
    TM3 = [GN_dict[PDBid][i] for i in TM3]
    TM6 = [GN_dict[PDBid][i] for i in TM6]
    TM7 = [GN_dict[PDBid][i] for i in TM7]
    return TM3,TM6,TM7

getmask("2I37")


#### 取得同时拥有active和inactive的受体名称，且分辨率>3.5

In [111]:
"""calculate receptors with both active and inactive
"""
import pandas as pd
data = pd.read_excel("./GPCRdb_checked_v2.xlsx",sheet_name="PDB_list")
pdb_list = pd.DataFrame(data)

dic = {item: [] for item in pdb_list["IUPHAR"].unique()} # dic["5-HT1A"] = []
for index,row in pdb_list.iterrows():
    if row["State"] in dic[row["IUPHAR"]] or float(row["Resolution"]) > 3.5:
        continue
    dic[row["IUPHAR"]].append(row["State"]) # dic["5-HT1A"] = ["Active","Inactive",...]
paired_receptors = [i for i in dic if "Active" in dic[i] and "Inactive" in dic[i]]


43


#### Way1: parse PDB from GPCRdb web (assigned with GN)

In [ ]:
import os
import json
import Bio
import pandas as pd
import numpy as np
import torch
from typing import List, Mapping, Optional, Dict
from absl import logging
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.MMCIFParser import MMCIFParser
import re
import glob
def bfactor_is_GN(bfactor: str) -> tuple(bool, str):
    """check if bfactor in PDB is GN

    Args:
        bfactor (str): bfactor in GPCRdb pdb

    Returns:
        bool: is GN or not
        bfactor: turn bafctor into GN (replace . to x)
    """
    if re.fullmatch("\d+\.\d+0$",bfactor): # in case we misadd the third 0
        bfactor = "%.2f"%float(bfactor)
    if not (check := re.search("^[1-8]\.\d+", bfactor)): # := in expression means assigned to the var 
        return False,bfactor
    bfactor = bfactor.replace(".","x")
    return True,bfactor
def get_GN_from_GPCRDB_PDB(pdb_file_pth: str, 
                       pdb_list: pd.DataFrame) -> torch.Tensor:
    if not os.path.exists(pdb_file_pth):
        raise ValueError('The directory does not exist!')
    # Obtain the name of the given PDB or MMCIF file 
    # such as: 1F88.pdb
    pdb_fname = os.path.basename(pdb_file_pth)

    if pdb_fname.endswith('.cif') or pdb_fname.endswith('.CIF'):
        parser = MMCIFParser()
    if pdb_fname.endswith('.pdb') or pdb_fname.endswith('.PDB'):
        parser =PDBParser()
    else:
        raise ValueError('This file is not a PDB or MMCIF file!')

    pdb_id = pdb_fname.split('.')[0][ :4].upper()
    structure = parser.get_structure(pdb_id, pdb_file_pth)
    models = list(structure.get_models())
    # If the number od models is not 1
    # raise error
    if len(models) > 1:
        print('Only PDB file with single model can be processed!')
        return (None, None)
    model = models[0]
    # Find the preferred chain for the given PDB ID in the pdb_list
    preferred_chains = pd.DataFrame(pdb_list, columns = ['PDB', 'Preferred chain']).set_index('PDB').to_dict('dict')['Preferred chain']
    target_chain_id = preferred_chains[pdb_id]
    chain = model[target_chain_id]
    # Find all residues of the preferred chain
    residues = list(chain.get_residues())
    GN2res = {}
    for res in residues:
        pos = res.get_id()[1]
        atoms = res['CA']
        b = atoms.get_bfactor()
        bfactor = str("%.3f"%atoms.get_bfactor()) # avoid 8.50 -> 8.5 or 8.551 -> 8.55
        if bfactor_is_GN(bfactor)[0] == True:
            GN2res[bfactor_is_GN(bfactor)[1]] = pos
    return GN2res
def loop(pdb_file_pth):
    GN_all = {}
    for i in glob.glob(f"{pdb_file_pth}/*"):
        pdb_id = os.path.split(i)[1].split(".")[0][:4].upper()
        GN_all[pdb_id] = get_GN_from_GPCRDB_PDB(i,pdb_list)
    with open('left_res_pos.json', 'w+') as f:
        f.write(json.dumps(GN_all, ensure_ascii = False, indent = 2))
#pdb_pth = "/home/yangsy/projects/multi-state/7rkn_GPCRDB.pdb"
csv_pth = "/home/yangsy/projects/multi-state/GPCRdb_checked_v2.xlsx"
pdb_list = pd.read_excel(csv_pth, sheet_name = 'PDB_list', dtype = {'Resolution': np.float16})
#loop("/home/yangsy/projects/multi-state/leftpdbs/")
print(bfactor_is_GN("5.500"))

#### Way2: get GN from GPCRDB API

In [ ]:
import numpy as np
import pandas as pd
import gpcrdb.structure as gstruc
from Bio.PDB.PDBParser import PDBParser
# get chain id
csv_pth = "/home/yangsy/projects/multi-state/GPCRdb_checked_v2.xlsx"
pdb_list = pd.read_excel(csv_pth, sheet_name = 'PDB_list', dtype = {'Resolution': np.float16})
preferred_chains = pd.DataFrame(pdb_list, columns = ['PDB', 'Preferred chain']).set_index('PDB').to_dict('dict')['Preferred chain']
target_chain_id = preferred_chains["2Y04"]
# get GN
p = PDBParser()
s = p.get_structure("2Y04", "/home/yangsy/projects/multi-state/raw_leftpdbs/2Y04.pdb")
data=gstruc.assign_generic_numbers(s,target_chain_id) # {res.full_id: GN}